In [ ]:
sys_prompt = """This is an excerpt of an academic paper.
Based on the input text generate a YAML containing: PHOTONIC, SiPHOTONIC, EXP, and CAT.
"PHOTONIC" should be "true" if the text is about a photonic device, system, circuit, or chip; otherwise "false".
"SiPHOTONIC" should be "true" if the text is about a silicon photonic device, system, circuit, or chip; otherwise "false".
"EXP" should be "true" is this is an experimental paper; and "false" if it is only a theoretical or numerical study.
"CAT" labels the application field of the work which could be TELECOM, DATACOM, SENSING, BIOLOGY, DISPLAY, or another single term to describe it best.

Your answer should be only four lines and strictly YAML formatted, as shown in the example below:
<example_output>
PHOTONIC: true
SiPHOTONIC: true
EXP: false
CAT: TELECOM
</example_output>
Don't preamble your output with phrases like "Here is the YAML data:" or "Here is the YAML output:".
"""

text = """
The continued growth in Hyperscale data center
(HDC) deployment is expected to drive world-wide Internet trafﬁc
to an astounding 21 zettabytes by 2021. This growth will place in-
creased demands on data center interconnects (DCIs) and drive the
capacity of the underlying electronic application speciﬁc integrated
circuit (ASIC) switch chips that route DCI ethernet trafﬁc, from
12.8 Tbps per chip today to 100 Tbps and beyond in the future.
This astounding growth will push the limits of today’s incoherent
ﬁber link technologies that connect switches, including power dis-
sipation, density, and practical engineering solutions. To overcome
these limits, high capacity coherent WDM, traditionally relegated
tothemetroandlong-haulnetworks,willneedtomoveintotheDCI.
However, migrating coherent WDM into the DCI, particularly for
link distances less than 2 km, will require elimination of power
consuming and costly technologies like the digital signal processor
(DSP). Additionally, new photonic integration technologies will be
needed to co-locate the coherent optical interfaces directly with
switch ASICs to alleviate the bandwidth, power, and density limits.
In this article, we introduce a new approach to DSP-free coher-
ent WDM for the DCI called FRESCO: FREquency Stabilized
Coherent Optical Links for Low Energy DCIs. FRESCO utilizes
spectrally pure, ultra-stable light source technology, normally as-
sociated with high-end scientiﬁc applications like atomic clocks, to
enable high capacity high-order WDM QAM with low bandwidth,
low power electronics normally associated with RF links. Terabits
per second FRESCO links based on shared, stabilized sources and
high-density coherent WDM silicon photonic coherent transceivers
that are co-located with the switch ASIC will pave the way to a
DSP-free coherent WDM scalable DCI solution.
"""

sys_prompt_json_schema = """You are a helpful assistant that outputs in JSON.
The use input is an academic text in photonic engineering. Generate JSON properties: PHOTONIC, SiPHOTONIC, EXP, and CAT.
"PHOTONIC" should be "true" if the text is about a photonic device, system, circuit, or chip; otherwise "false".
"SiPHOTONIC" should be "true" if the text is about a silicon photonic device, system, circuit, or chip; otherwise "false".
"EXP" should be "true" is this is an experimental paper; and "false" if it is only a theoretical or numerical study.
"CAT" labels the application field of the work which could be TELECOM, DATACOM, SENSING, BIOLOGY, DISPLAY, or another single term to describe it best.
"""

json_schema = {
    "type": "json_object",
    "schema": {
        "type": "object",
        "properties": {
            "PHOTONIC": {"type": "boolean"},
            "SiPHOTONIC": {"type": "boolean"},
            "EXP": {"type": "boolean"},
            "CAT": {"type": "string"},
        },
        "required": ["PHOTONIC", "SiPHOTONIC", "EXP", "CAT"],
    },
}

from openai import OpenAI

client = OpenAI(
    base_url="https://photonic.pagekite.me/v1",
    api_key="QMKyK16tQGRjkKUjPfeZzlcUOIRrdH8U",
)

In [ ]:
# simple chat

response = client.chat.completions.create(
    model="gpt-xxx",
    temperature=0.1,
    max_tokens=300,
    messages=[
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": text},
    ],
)
print(response.choices[0].message.content)

In [ ]:
# chat in json schema mode

response = client.chat.completions.create(
    model="gpt-xxx",
    temperature=0.1,
    max_tokens=50,
    messages=[
        {
            "role": "system",
            "content": sys_prompt_json_schema,
        },
        {"role": "user", "content": text},
    ],
    response_format=json_schema,
)
print(response.choices[0].message.content)

In [ ]:
# using pydantic


import json

from groq import Groq
from pydantic import BaseModel

groq = Groq()


# Data model for LLM to generate
class Ingredient(BaseModel):
    name: str
    quantity: str
    quantity_unit: str | None


class Recipe(BaseModel):
    recipe_name: str
    ingredients: list[Ingredient]
    directions: list[str]


def get_recipe(recipe_name: str) -> Recipe:
    chat_completion = groq.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a recipe database that outputs recipes in JSON.\n"
                # Pass the json schema to the model. Pretty printing improves results.
                f" The JSON object must use the schema: {json.dumps(Recipe.model_json_schema(), indent=2)}",
            },
            {
                "role": "user",
                "content": f"Fetch a recipe for {recipe_name}",
            },
        ],
        model="llama3-8b-8192",
        temperature=0,
        # Streaming is not supported in JSON mode
        stream=False,
        # Enable JSON mode by setting the response format
        response_format={"type": "json_object"},
    )
    return Recipe.model_validate_json(chat_completion.choices[0].message.content)


def print_recipe(recipe: Recipe):
    print("Recipe:", recipe.recipe_name)

    print("\nIngredients:")
    for ingredient in recipe.ingredients:
        print(
            f"- {ingredient.name}: {ingredient.quantity} {ingredient.quantity_unit or ''}"
        )
    print("\nDirections:")
    for step, direction in enumerate(recipe.directions, start=1):
        print(f"{step}. {direction}")


recipe = get_recipe("apple pie")
print_recipe(recipe)